## Check List 4 - MotionFilter
* Motion filters are quick decision makers that prevents trying to plan a infeasible motion.
* Here, ReachChecker is tested, compared with actual planning results in random trials
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4-A.3 LatticedChecker**  
  - pick, place
    - single process
    - *multi process is not supported with LatticedChecker, which uses GPU*
    
* You need trained model to use ReachChecker
* To use **LatticedChecker**, get model files from https://github.com/rnb-disinfection/arxiv-highcap/tree/rnb-planning/model/latticized and put them in *$RNB_PLANNING_DIR/model/latticized* folder.
* For robots that are indy7 variants (indy7gripper...), copy indy7 model file and assign the robot name as the file name.

**[Trouble Shooting]**
```
LatticedChecker uses tensorflow model on separate python3 program, through a shared memory.
If some error raises with shared memory, try uncomment and run below cell, to reset shared memory.
```

In [1]:
# import SharedArray as sa
# robot_type_name = "indy7"
# sa.delete("shm://{}.grasp_img".format(robot_type_name))
# sa.delete("shm://{}.arm_img".format(robot_type_name))
# sa.delete("shm://{}.rh_mask".format(robot_type_name))
# sa.delete("shm://{}.result".format(robot_type_name))
# sa.delete("shm://{}.query_in".format(robot_type_name))
# sa.delete("shm://{}.response_out".format(robot_type_name))
# sa.delete("shm://{}.query_quit".format(robot_type_name))

## set running directory

In [2]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [3]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir

In [4]:
VISUALIZE = True
CLEARANCE = 1e-3

# ROBOT_TYPE = RobotType.indy7gripper
# ROBOT_NAME = "indy0"
# TOOL_XYZ = (0,0,0.14)
# TOOL_RPY = (-np.pi/2,0,0)
# GRIP_DEPTH = 0.05
# HOME_POSE = (0,0,0,0,0,0)

ROBOT_TYPE = RobotType.panda
ROBOT_NAME = "panda0"
TOOL_XYZ = (0,0,0.112)
TOOL_RPY = (-np.pi/2,0,0)
GRIP_DEPTH = 0.03
HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)

## init combined robot config

In [5]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                None)]
              , connection_list=[False])
TOOL_LINK = crob.get_robot_tip_dict().values()[0]
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")

connection command:
panda0: False


## get ghnd with detected robot config

In [6]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


In [7]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [8]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                   dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## planner
* Initiailizaing *LatticedChecker* can take ~ 1 min.

In [12]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
mplan = MoveitPlanner(pscene)
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
lcheck = LatticedChecker(pscene, gcheck)
checkers_all = [gcheck, rcheck, lcheck]

In [13]:
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

# Object Classes

In [14]:
from pkg.utils.gjk import get_point_list, get_gjk_distance
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask
from pkg.planning.filtering.lattice_model.scene_building import *

### Test: pick&place motions

##### test few times with visualization

In [15]:
from pkg.planning.constraint.constraint_common import calc_redundancy
import random

Nmax_obj = 3
VISUALIZE = True
result_dict = defaultdict(list)
N_TRIAL = 5
i_count = 0
MP_TIMEOUT = 1

while i_count < N_TRIAL:
    pscene.clear_subjects()
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    initial_state = pscene.initialize_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    tplan.prepare()
    
    scenario = "pick"
    from_state=initial_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("grip0",)+("wp",)*len(pscene.subject_name_list[1:])
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)

    
    for _ in range(10): # try multiple times to get available put position
        to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
        obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]: # test gcheck anc rcheck
            success_mfilter = mfilter.check(to_state.binding_state["obj_0"], Q_dict)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    i_count += 1
        
    # test lcheck
    success_mfilter = lcheck.check(to_state.binding_state["obj_0"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.01)
        pick_state = pscene.rebind_all(
            [to_state.binding_state[sname].get_chain() for sname in pscene.subject_name_list], LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    from_state=pick_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("gp",)+("wp",)*len(pscene.subject_name_list[1:])
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    
    for _ in range(10): # try multiple times to get available put position
        to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
        obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]: # test gcheck anc rcheck
            success_mfilter = mfilter.check(to_state.binding_state["obj_0"], Q_dict)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    # test lcheck
    success_mfilter = lcheck.check(to_state.binding_state["obj_0"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.01)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

1/5: Result match for pick: True / True
2/5: Result mismatch for pick: True / False
3/5: Result match for pick: True / True
3/5: Result match for place: False / False
4/5: Result match for pick: True / True
5/5: Result match for pick: True / True
5/5: Result match for place: True / True


##### test many times without visualization

In [16]:
from pkg.planning.constraint.constraint_common import calc_redundancy
import random

Nmax_obj = 3
VISUALIZE = False
result_dict = defaultdict(list)
N_TRIAL = 50
i_count = 0
MP_TIMEOUT = 1

while i_count < N_TRIAL:
    pscene.clear_subjects()
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    initial_state = pscene.initialize_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    tplan.prepare()
    
    scenario = "pick"
    from_state=initial_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("grip0",)+("wp",)*len(pscene.subject_name_list[1:])
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)

    
    for _ in range(10): # try multiple times to get available put position
        to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
        obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]: # test gcheck anc rcheck
            success_mfilter = mfilter.check(to_state.binding_state["obj_0"], Q_dict)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    i_count += 1
        
    # test lcheck
    success_mfilter = lcheck.check(to_state.binding_state["obj_0"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
        pick_state = pscene.rebind_all(
            [to_state.binding_state[sname].get_chain() for sname in pscene.subject_name_list], LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    from_state=pick_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("gp",)+("wp",)*len(pscene.subject_name_list[1:])
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    
    for _ in range(10): # try multiple times to get available put position
        to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
        obj, handle, actor = to_state.binding_state["obj_0"].get_instance_chain(pscene)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        for mfilter in [gcheck, rcheck]: # test gcheck anc rcheck
            success_mfilter = mfilter.check(to_state.binding_state["obj_0"], Q_dict)
            if not success_mfilter:
                break
        if success_mfilter:
            break
    if not success_mfilter:
        continue
        
    # test lcheck
    success_mfilter = lcheck.check(to_state.binding_state["obj_0"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.05)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

1/50: Result match for pick: True / True
2/50: Result match for pick: True / True
3/50: Result match for pick: True / True
4/50: Result match for pick: True / True
5/50: Result match for pick: True / True
6/50: Result match for pick: True / True
7/50: Result match for pick: True / True
7/50: Result match for place: True / True
8/50: Result match for pick: True / True
9/50: Result match for pick: True / True
10/50: Result match for pick: True / True
10/50: Result match for place: False / False
11/50: Result match for pick: True / True
11/50: Result match for place: True / True
12/50: Result match for pick: True / True
13/50: Result match for pick: True / True
14/50: Result match for pick: True / True
14/50: Result match for place: False / False
15/50: Result mismatch for pick: True / False
16/50: Result match for pick: True / True
17/50: Result match for pick: True / True
18/50: Result match for pick: True / True
18/50: Result match for place: False / False
19/50: Result match for pick:

In [17]:
ACCURACY_CUT = 80
print("Accuracy cutline: {} %".format(ACCURACY_CUT))
for k, v in sorted(result_dict.items()):
    v_arr = np.array(v)
    suc_idx = np.where(v_arr[:, 1])[0]
    fail_idx = np.where(np.logical_not(v_arr[:, 1]))[0]
    acc_val = np.round(np.mean(v_arr[:,0]==v_arr[:,1])*100, 1)
    acc_suc = np.round(np.mean(v_arr[suc_idx,0]==v_arr[suc_idx,1])*100, 1)
    acc_fail = np.round(np.mean(v_arr[fail_idx,0]==v_arr[fail_idx,1])*100, 1)
    print("="*60)
    print("{} accuracy: {} % / success accuracy: {} % / fail accuracy: {} % / success count:{}/{}".format(k, acc_val, acc_suc, acc_fail, len(suc_idx), len(v_arr)))
    assert acc_val>ACCURACY_CUT and acc_suc>ACCURACY_CUT, \
            "Accuracy {}/{} is lower than {}. Consider debugging the algorithm.".format(acc_val, acc_suc, ACCURACY_CUT)
    print("""OK with "{}" scenario""".format(k))

Accuracy cutline: 80 %
pick accuracy: 90.0 % / success accuracy: 100.0 % / fail accuracy: 16.7 % / success count:44/50
OK with "pick" scenario
place accuracy: 100.0 % / success accuracy: 100.0 % / fail accuracy: 100.0 % / success count:7/11
OK with "place" scenario


### planning: pick&place

In [20]:
from pkg.planning.constraint.constraint_common import calc_redundancy
from copy import deepcopy

pscene.clear_subjects()
    
Nmax_obj = 1
gtimer = GlobalTimer.instance()
gtimer.reset()
## add floor, ceiling
floor = Floor(gscene, "floor")
gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                   dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                   color=floor.geometry.color, display=True, collision=True, fixed=True)
# ceiling = Ceiling(gscene, "ceiling")

## set workplane
wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

gp_colliding = True
while gp_colliding:
    ## set goalplane
    gp = WorkPlane(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
    gp_colliding = wp.is_overlapped_with(gp.geometry)
pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


## add object
obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH)

obj_set_list = []
for i_obj, obj in enumerate(obj_list):
    obj_pscene, handles = add_object(pscene, obj)
    obj_set_list.append((obj, obj_pscene, handles))
obj, obj_pscene, handles = obj_set_list[0]

obj_pscene.geometry.color = (0.8,0.2,0.2,1)
if VISUALIZE:
    gscene.set_rviz()

mplan.update_gscene()
initial_state = pscene.initialize_state(HOME_POSE)
pscene.set_object_state(initial_state)
from_state = initial_state.copy(pscene)

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

gtimer = GlobalTimer.instance()
gtimer.reset()

mplan.motion_filters = checkers_all
goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
mplan.reset_log(True)
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
              timeout_loop=10, multiprocess=True, timeout=1, add_homing=False)
elapsed = gtimer.toc("plan")/1000
schedule = tplan.get_best_schedule(at_home=False)
res = len(schedule)>0
if res:
    ppline.play_schedule(schedule)
else:
    raise(RuntimeError("Failed to make plan - may be impossible problem was generated. try again up to 10 times or debug"))

Use 18/36 agents
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try: 0 - ('wp',)->('grip0',)
error: 2.9
try transition motion
error: 2.3
try transition motion
transition motion tried: True
result: 0 - ('wp',)->('grip0',) = success
branching: 0->1 (0.29/10.0 s, steps/err: 35(128.192186356 ms)/0.00100235332318)
try: 1 - ('grip0',)->('gp',)
Motion Filer Failure: ReachChecker
try: 0 - ('wp',)->('grip0',)
result: 1 - ('grip0',)->('gp',) = fail
try: 1 - ('grip0',)->('wp',)
try: 1 - ('grip0',)->('wp',)
try: 1 - ('grip0',)->('gp',)
error: 2.9
error: 0.35
Motion Filer Failure: ReachChecker
result: 1 - ('grip0',)->('gp',) = fail
try transition motion
try transition motion
error: 1.4
try transition motion
transition motion tried: True
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('

error: 0.31
result: 1 - ('grip0',)->('wp',) = fail
result: 4 - ('wp',)->('grip0',) = success
Motion Filer Failure: ReachChecker
try: 5 - ('grip0',)->('wp',)
try transition motion
try: 2 - ('grip0',)->('gp',)
result: 7 - ('wp',)->('grip0',) = fail
try: 10 - ('grip0',)->('gp',)
result: 1 - ('grip0',)->('wp',) = fail
error: 2.8
branching: 4->10 (0.85/10.0 s, steps/err: 21(288.26713562 ms)/0.00142175272925)
try transition motion
try: 0 - ('wp',)->('grip0',)
result: 1 - ('grip0',)->('wp',) = fail
try: 1 - ('grip0',)->('gp',)
try: 2 - ('grip0',)->('wp',)
try: 5 - ('grip0',)->('gp',)
transition motion tried: True
Motion Filer Failure: ReachChecker
transition motion tried: True
Motion Filer Failure: ReachChecker
Motion Filer Failure: ReachChecker
result: 5 - ('grip0',)->('wp',) = success
error: 0.35
result: 2 - ('grip0',)->('gp',) = fail
branching: 5->11 (0.88/10.0 s, steps/err: 28(227.761030197 ms)/0.00141553519948)
result: 10 - ('grip0',)->('gp',) = fail
result: 4 - ('wp',)->('grip0',) = suc

branching: 3->22 (1.17/10.0 s, steps/err: 65(629.068851471 ms)/0.00105125462091)
error: 0.82
try transition motion
try: 22 - ('grip0',)->('gp',)
transition motion tried: True
Motion Filer Failure: ReachChecker
result: 22 - ('grip0',)->('gp',) = fail
result: 2 - ('grip0',)->('wp',) = success
branching: 2->23 (1.18/10.0 s, steps/err: 111(312.252998352 ms)/0.0012801471998)
try: 22 - ('grip0',)->('gp',)
transition motion tried: True
try: 16 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 22 - ('grip0',)->('gp',) = fail
error: 0.88
try transition motion
Motion Filer Failure: GraspChecker
result: 5 - ('grip0',)->('wp',) = success
try: 5 - ('grip0',)->('gp',)
result: 16 - ('wp',)->('grip0',) = fail
Motion Filer Failure: ReachChecker
branching: 5->24 (1.2/10.0 s, steps/err: 21(265.70391655 ms)/0.00100509472236)
result: 5 - ('grip0',)->('gp',) = fail
try: 1 - ('grip0',)->('gp',)
try: 16 - ('wp',)->('grip0',)
try: 4 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
Motion

try: 4 - ('wp',)->('grip0',)
try: 2 - ('grip0',)->('gp',)
try: 4 - ('wp',)->('grip0',)
error: 2.8
Motion Filer Failure: ReachChecker
Motion Filer Failure: GraspChecker
result: 4 - ('wp',)->('grip0',) = fail
try transition motion
try: 14 - ('grip0',)->('wp',)
Motion Filer Failure: ReachChecker
result: 2 - ('grip0',)->('gp',) = fail
result: 14 - ('grip0',)->('wp',) = fail
try: 17 - ('wp',)->('grip0',)
try: 23 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
Motion Filer Failure: GraspChecker
result: 17 - ('wp',)->('grip0',) = fail
result: 23 - ('wp',)->('grip0',) = fail
try: 12 - ('grip0',)->('wp',)
error: 2.5
transition motion tried: True
try: 25 - ('wp',)->('grip0',)
try transition motion
transition motion tried: True
Motion Filer Failure: ReachChecker
result: 1 - ('grip0',)->('wp',) = success
result: 12 - ('grip0',)->('wp',) = fail
Goal reached
try: 31 - ('grip0',)->('gp',)
Motion Filer Failure: GraspChecker
result: 12 - ('grip0',)->('gp',) = success
branching: 12->35 (1.45/10

error: 2.8
try: 27 - ('wp',)->('grip0',)
try: 17 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
try transition motion
try: 46 - ('grip0',)->('wp',)
result: 47 - ('grip0',)->('gp',) = fail
Motion Filer Failure: ReachChecker
try: 29 - ('grip0',)->('gp',)
transition motion tried: True
error: 1.7
result: 46 - ('grip0',)->('gp',) = fail
try transition motion
try: 32 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
Motion Filer Failure: ReachChecker
result: 12 - ('grip0',)->('wp',) = success
error: 1.6
result: 29 - ('grip0',)->('gp',) = fail
branching: 12->48 (1.73/10.0 s, steps/err: 54(205.857038498 ms)/0.00179904555535)
try transition motion
result: 32 - ('wp',)->('grip0',) = fail
try: 48 - ('wp',)->('grip0',)
try: 41 - ('grip0',)->('wp',)
error: 1.3
transition motion tried: True
Motion Filer Failure: GraspChecker
transition motion tried: True
try transition motion
try: 29 - ('grip0',)->('wp',)
result: 20 - ('wp',)->('grip0',) = success
result: 1 - ('grip0',)->('wp',) = s

result: 54 - ('grip0',)->('gp',) = fail
try: 49 - ('grip0',)->('gp',)
try: 47 - ('grip0',)->('wp',)
result: 58 - ('grip0',)->('gp',) = fail
Motion Filer Failure: ReachChecker
transition motion tried: True
try transition motion
branching: 38->62 (2.02/10.0 s, steps/err: 47(188.899040222 ms)/0.000872640469575)
error: 0.99
result: 62 - ('grip0',)->('gp',) = fail
Motion Filer Failure: ReachChecker
try: 56 - ('grip0',)->('wp',)
try: 49 - ('grip0',)->('wp',)
Motion Filer Failure: ReachChecker
try: 27 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
transition motion tried: True
result: 46 - ('grip0',)->('wp',) = success
branching: 46->63 (2.04/10.0 s, steps/err: 14(204.636096954 ms)/0.000885952198646)
result: 56 - ('grip0',)->('wp',) = fail
result: 49 - ('grip0',)->('gp',) = fail
try transition motion
try: 43 - ('grip0',)->('wp',)
try: 44 - ('wp',)->('grip0',)
result: 47 - ('grip0',)->('wp',) = fail
try: 34 - ('wp',)->('grip0',)
result: 23 - ('wp',)->('grip0',) = success
branching: 2

Motion Filer Failure: ReachChecker
transition motion tried: True
result: 72 - ('grip0',)->('gp',) = fail
result: 44 - ('wp',)->('grip0',) = success
result: 29 - ('grip0',)->('gp',) = fail
Motion Filer Failure: ReachChecker
result: 10 - ('grip0',)->('gp',) = fail
result: 46 - ('grip0',)->('gp',) = fail
branching: 44->73 (2.33/10.0 s, steps/err: 65(268.009901047 ms)/0.00109802424822)
error: 1.8
try: 73 - ('grip0',)->('gp',)
try: 22 - ('grip0',)->('wp',)
try: 51 - ('grip0',)->('wp',)
result: 56 - ('grip0',)->('wp',) = success
branching: 56->74 (2.35/10.0 s, steps/err: 6(247.353076935 ms)/0.00150581376065)
transition motion tried: True
transition motion tried: True
try transition motion
result: 19 - ('grip0',)->('wp',) = success
try: 49 - ('grip0',)->('wp',)
transition motion tried: True
Motion Filer Failure: ReachChecker
try: 56 - ('grip0',)->('gp',)
Motion Filer Failure: GraspChecker
result: 51 - ('grip0',)->('wp',) = fail
error: 0.76
try: 62 - ('grip0',)->('wp',)
result: 17 - ('wp',)->(

result: 49 - ('grip0',)->('wp',) = success
branching: 49->87 (2.6/10.0 s, steps/err: 67(237.954139709 ms)/0.00103554838284)
error: 2.7
try transition motion
error: 0.91
try transition motion
transition motion tried: True
result: 62 - ('grip0',)->('wp',) = success
branching: 62->88 (2.64/10.0 s, steps/err: 69(273.79488945 ms)/0.00109216951113)
error: 1.8
try transition motion
transition motion tried: True
result: 5 - ('grip0',)->('wp',) = success
branching: 5->89 (2.65/10.0 s, steps/err: 73(257.951021194 ms)/0.00138130434153)
transition motion tried: True
error: 1.3
result: 41 - ('grip0',)->('wp',) = success
branching: 41->90 (2.66/10.0 s, steps/err: 42(265.682935715 ms)/0.001397302558)
try transition motion
transition motion tried: True
result: 43 - ('grip0',)->('wp',) = success
branching: 43->91 (2.67/10.0 s, steps/err: 57(263.017892838 ms)/0.00126334190264)
transition motion tried: True
Goal reached
result: 51 - ('grip0',)->('gp',) = success
branching: 51->92 (2.69/10.0 s, steps/err:

========================== FINISHED (10.2 / 10.0 s) ==============================]
('wp',)->('grip0',)
('grip0',)->('gp',)


In [21]:
ppline.play_schedule(schedule)

('wp',)->('grip0',)
('grip0',)->('gp',)
